In [ ]:
# download ESACCI L4 SSS 
#! PYTHONPATH=../../pycommon ./get_l4_sss_esacci.py \
#         --start_date 20180101 --end_date 20191231 \
#         --log 


In [ ]:
# generate interpolating weights from latlon grids of WOA18 to MOM6 tripolar grids

#! pwd
#! ../woa/gen_regridder_woa18.py wts_woa18_to_mom_p25.nc \
#--target_grid_path ../../test_data/flood_salt_restore_PHC2.1440x1080.v20180405.nc \
#--source_grid_path ../../test_data/woa18_decav_s01_04.nc

In [ ]:
# generate interpolating weights from latlon grids of L4 SSS to WOA18 grids

#! pwd
#! ./gen_regridder_esacci_to_woa18.py wts_l4sss_to_woa18.nc \
#--target_grid_path ../../test_data/woa18_decav_s01_04.nc \
#--source_grid_path l4_sss_esacci/2019/201901/20190101/ESACCI-SEASURFACESALINITY-L4-SSS-MERGED_OI_7DAY_RUNNINGMEAN_DAILY_25km-20190101-fv3.21.nc

In [ ]:
# generate 12-month salinity files from WOA18

! PYTHONPATH=../../pycommon ../woa/fill_sss_woa18_12months.py ../../test_data/woa18 \
--start_month 1 --end_month 12

In [ ]:
# create daily merged L4&WOA SSS on WOA18 grids

! time PYTHONPATH=../../pycommon ./merge_l4_woa18.py \
        --regridder_l4_to_woa_path wts_l4sss_to_woa18.nc \
        --regridder_woa_to_mom_path wts_woa18_to_mom_p25.nc \
        --woa_grid_path ../../test_data/woa18_decav_s01_04.nc \
        --l4_grid_path l4_sss_esacci/2019/201901/20190101/ESACCI-SEASURFACESALINITY-L4-SSS-MERGED_OI_7DAY_RUNNINGMEAN_DAILY_25km-20190101-fv3.21.nc \
        --mom_grid_path ../../test_data/flood_salt_restore_PHC2.1440x1080.v20180405.nc \
        --mom_lsmsk_path ../../test_data/ocean_topog.nc \
        --woa18_filled_path ../woa/sss_woa18_10m/filled_sss_s01_e12.nc \
        --l4_sss_path l4_sss_esacci/2019/201904/20190430/ESACCI-SEASURFACESALINITY-L4-SSS-MERGED_OI_7DAY_RUNNINGMEAN_DAILY_25km-20190430-fv3.21.nc \
        --radius 10 \
        --output_file_name test_l4_1day.nc

In [ ]:
# modify the script that loops over all days
#

! time PYTHONPATH=../../pycommon ./loop_sal_esacci_daily.py 

In [ ]:
# merge daily sal into yearly sal restore file

! time PYTHONPATH=../../pycommon ./create_sal_restore_day2year.py \
        --template_file_path template.1440x1080.v20180405.nc \
        --start_date 20190101 \
        --end_date 20191231 \
        --daily_sal_dir remapped_l4_sss \
        --daily_sal_file_name  sss_remapped_YYYYMMDD_radius010.nc \
        --output_file_name salinity_restore_woa18_L4_SSS_ESACCI_s20190101_e20191231.nc 

In [ ]:
import datetime as dt
import calendar
import os, sys
import numpy as np
from netCDF4 import Dataset

import matplotlib.pyplot as plt

import cartopy as cart
import cartopy.feature
from cmocean import cm as cmo
%matplotlib inline

# check the salinity field from the daily restore file
if False:
    
    ds_path = 'test_l4_1day.nc'
    f = Dataset(ds_path)
    lon1d_woa = f.variables["lon1d_woa"][:]
    lat1d_woa = f.variables["lat1d_woa"][:]
    lon2d_mom = f.variables["lon2d_mom"][:]
    lat2d_mom = f.variables["lat2d_mom"][:]
    sss_unfilled = f.variables["sss_unfilled"][:]
    sss_filled_step1 = f.variables["sss_filled_step1"][:]
    sss_filled_step2 = f.variables["sss_filled_step2"][:]
    sss_remapped_mom = f.variables["sss_remapped_mom"][:]
    f.close()

    clevs = np.linspace(32.,38.5, 21)
    clevs2 = np.linspace(-0.5, 0.5, 21)

    projPC = cart.crs.PlateCarree(central_longitude=-180)

    fig = plt.figure( figsize=(16,9))

    ax1 = plt.subplot(2,2,1, projection=projPC)
    gl1 = ax1.gridlines(draw_labels=True, linewidth=2, color='gray', alpha=0.1, linestyle='--')
    gl1.xlabels_top = False
    ax1.set_title("sss_unfilled")
    ax1.coastlines(resolution='110m',     color='black', ls='-', alpha=1.)
    ax1.add_feature(cartopy.feature.LAND,  edgecolor='k', facecolor=("lightgray"))
    im1=ax1.contourf(lon1d_woa, lat1d_woa, sss_unfilled, \
                 transform=cart.crs.PlateCarree(),\
                 levels=clevs, cmap=plt.cm.jet, extend="both")
    plt.colorbar(im1, shrink=0.5)


#
    ax2 = plt.subplot(2,2,2, projection=projPC)
    gl2 = ax2.gridlines(draw_labels=True, linewidth=2, color='gray', alpha=0.1, linestyle='--')
    gl2.xlabels_top = False
    ax2.set_title("filled_step1")
    ax2.coastlines(resolution='110m',     color='black', ls='-', alpha=1.)
    ax2.add_feature(cartopy.feature.LAND, edgecolor='k', facecolor=("lightgray"))
    im2=ax2.contourf(lon1d_woa, lat1d_woa, sss_filled_step1,  \
                 transform=cart.crs.PlateCarree(),\
                 levels=clevs, cmap=plt.cm.jet, extend="both")
    plt.colorbar(im2, shrink=0.5)


#
    ax3 = plt.subplot(2,2,3, projection=projPC)
    gl3 = ax3.gridlines(draw_labels=True, linewidth=2, color='gray', alpha=0.1, linestyle='--')
    gl3.xlabels_top = False
    ax3.set_title("filled_step2")
    ax3.coastlines(resolution='110m',     color='black', ls='-', alpha=1.)
    ax3.add_feature(cartopy.feature.LAND, edgecolor='k', facecolor=("lightgray"))
    im3=ax3.contourf(lon1d_woa, lat1d_woa, sss_filled_step2,   \
                 transform=cart.crs.PlateCarree(),\
                 levels=clevs, cmap=plt.cm.jet, extend="both")
    plt.colorbar(im3, shrink=0.5)
    
#
    ax4 = plt.subplot(2,2,4, projection=projPC)
    gl4 = ax4.gridlines(draw_labels=True, linewidth=2, color='gray', alpha=0.1, linestyle='--')
    gl4.xlabels_top = False
    ax4.set_title("sss_remapped_to_tripolar")
    ax4.coastlines(resolution='110m',     color='black', ls='-', alpha=1.)
    ax4.add_feature(cartopy.feature.LAND, edgecolor='k', facecolor=("lightgray"))
    im4=ax4.contourf(lon2d_mom, lat2d_mom, sss_remapped_mom,   \
                 transform=cart.crs.PlateCarree(),\
                 levels=clevs, cmap=plt.cm.jet, extend="both")
    plt.colorbar(im4, shrink=0.5)    
    